# Fantasy Football Analysis - 2019

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml

In [166]:
url = 'https://www.pro-football-reference.com/years/2018/fantasy.htm'

In [167]:
r = requests.get(url)

In [168]:
soup = BeautifulSoup(r.text, 'lxml')

In [169]:
rows = soup.find('table',{'id':'fantasy'}).tbody.find_all('tr')

In [170]:
player_stats = []
for row in rows:
    player_dict = {}
    for td in row:
        try:
            player_dict[td['data-stat']]=td.text
        except:
            continue
    player_stats.append(player_dict)        

In [190]:
df = pd.DataFrame(player_stats)
df['player'] = df.player.str.replace('*','')
df['player'] = df.player.str.replace('+','')

In [191]:
df.loc[:,headers].head()

,fantasy_pos,player,team,fantasy_rank_pos,g,pass_yds,pass_td,pass_int,rush_yds,rush_td,rec_yds,rec_td,two_pt_pass,two_pt_md
0,RB,Todd Gurley,LAR,1,14,0,0,0,1251,17,580,4,,3
1,RB,Saquon Barkley,NYG,2,16,0,0,0,1307,11,721,4,,1
2,RB,Christian McCaffrey,CAR,3,16,50,1,0,1098,7,867,6,,
3,RB,Alvin Kamara,NOR,4,15,0,0,0,883,14,709,4,,3
4,QB,Patrick Mahomes,KAN,1,16,5097,50,12,272,2,0,0,,1


In [192]:
headers = ['fantasy_pos','player','team','fantasy_rank_pos','g','pass_yds','pass_td','pass_int','rush_yds','rush_td','rec_yds','rec_td','two_pt_pass','two_pt_md']

In [193]:
rb = df.loc[df['fantasy_pos']=='RB', headers]

In [194]:
wr = df.loc[df['fantasy_pos']=='WR', headers]

In [195]:
qb = df.loc[df['fantasy_pos']=='QB', headers]

In [196]:
te = df.loc[df['fantasy_pos']=='TE', headers]

In [197]:
df_list = [rb, wr, qb, te]

In [198]:
def empty(x):
    if not x:
        return 0.0
    else:
        return float(x)
    
passing_yards = (lambda x: empty(x)*0.05)
passing_td = (lambda x: empty(x)*6.0)
passing_int = (lambda x: empty(x)*-2.0)
rushing_yards = (lambda x: empty(x)*0.1)
rushing_td = (lambda x: empty(x)*6.0)
receiving_yards= (lambda x: empty(x)*0.1)
receiving_td = (lambda x: empty(x)*6.0)
scoring_2pt = (lambda x: empty(x)*2.0)

In [199]:
for d in df_list:
    d['pts_pass_yds'] = d.pass_yds.apply(passing_yards)
    d['pts_pass_td'] = d.pass_td.apply(passing_td)
    d['pts_pass_int'] = d.pass_int.apply(passing_int)
    d['pts_rec_yds'] = d.rec_yds.apply(receiving_yards)
    d['pts_rec_td'] = d.rec_td.apply(receiving_td)
    d['pts_rush_ytd'] = d.rush_yds.apply(rushing_yards)
    d['pts_rush_td'] = d.rush_td.apply(rushing_td)
    d['pts_two_pt_pass'] = d.two_pt_pass.apply(scoring_2pt)
    d['pts_two_pt_md'] = d.two_pt_md.apply(scoring_2pt)
    d['pts_total'] = (d.pts_pass_yds + d.pts_pass_td + d.pts_pass_int + d.pts_rec_yds + d.pts_rec_yds + d.pts_rec_td + d.pts_rush_ytd + d.pts_rush_td + d.pts_two_pt_pass + d.pts_two_pt_md)
    d['pts_per_game'] = d.pts_total / d.g.astype('float')

In [217]:
qb = qb.loc[qb.player!='Andrew Luck']

In [225]:
qb.sort_values('pts_per_game', ascending=False).head(10)

,fantasy_pos,player,team,fantasy_rank_pos,g,pass_yds,pass_td,pass_int,rush_yds,rush_td,...,pts_pass_td,pts_pass_int,pts_rec_yds,pts_rec_td,pts_rush_ytd,pts_rush_td,pts_two_pt_pass,pts_two_pt_md,pts_total,pts_per_game
4,QB,Patrick Mahomes,KAN,1,16,5097,50,12,272,2,...,300.0,-24.0,0.0,0.0,27.2,12.0,0.0,2.0,572.05,35.753125
20,QB,Matt Ryan,ATL,2,16,4924,35,7,125,3,...,210.0,-14.0,0.5,6.0,12.5,18.0,4.0,0.0,483.70,30.231250
26,QB,Ben Roethlisberger,PIT,3,16,5129,34,16,98,3,...,204.0,-32.0,-0.1,0.0,9.8,18.0,8.0,0.0,464.05,29.003125
97,QB,Ryan Fitzpatrick,TAM,28,8,2366,17,12,152,2,...,102.0,-24.0,0.0,0.0,15.2,12.0,2.0,0.0,225.50,28.187500
52,QB,Drew Brees,NOR,8,15,3992,32,5,22,4,...,192.0,-10.0,0.1,0.0,2.2,24.0,2.0,0.0,410.00,27.333333
31,QB,Deshaun Watson,HOU,4,16,4165,26,9,551,5,...,156.0,-18.0,0.0,0.0,55.1,30.0,0.0,0.0,431.35,26.959375
45,QB,Jared Goff,LAR,7,16,4688,32,12,108,2,...,192.0,-24.0,0.0,0.0,10.8,12.0,6.0,0.0,431.20,26.950000
80,QB,Cam Newton,CAR,12,14,3395,24,13,488,4,...,144.0,-26.0,0.0,0.0,48.8,24.0,4.0,0.0,364.55,26.039286
43,QB,Aaron Rodgers,GNB,6,16,4442,25,2,269,2,...,150.0,-4.0,0.0,0.0,26.9,12.0,4.0,2.0,413.00,25.812500
58,QB,Russell Wilson,SEA,9,16,3448,35,7,376,0,...,210.0,-14.0,-1.1,0.0,37.6,0.0,2.0,0.0,405.80,25.362500


In [226]:
qb.to_excel('qb_stats.xlsx')

In [227]:
rb.to_excel('rb_stats.xlsx')

In [228]:
wr.to_excel('wr_stats.xlsx')

In [229]:
te.to_excel('te_stats.xlsx')